In [1]:
import findspark

findspark.find()
findspark.init()
import pyspark

In [2]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel

In [3]:
sc = pyspark.SparkContext(appName="K-means_MLlib")

21/08/26 12:10:06 WARN Utils: Your hostname, DESKTOP-HG8PKN0 resolves to a loopback address: 127.0.1.1; using 172.31.145.95 instead (on interface eth0)
21/08/26 12:10:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/26 12:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Load and parse the data
data = sc.textFile("data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

In [5]:
data.collect()

['0.0 0.0 0.0',
 '0.1 0.1 0.1',
 '0.2 0.2 0.2',
 '9.0 9.0 9.0',
 '9.1 9.1 9.1',
 '9.2 9.2 9.2']

In [6]:
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

21/08/26 12:10:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/26 12:10:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [7]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [8]:
WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 0.6928203230275529


In [9]:
# Save and load model
# clusters.save(sc, "/home/kira/tmp/Spark/PythonKMeansExample/KMeansModel2")
# sameModel = KMeansModel.load(sc, "/home/kira/tmp/Spark/PythonKMeansExample/KMeansModel2")

In [ ]:
# now shut down the Spark context
sc.stop()